In [1]:
import yfinance as yf
import numpy as np
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_optimization.algorithms import MinimumEigenOptimizer

stocks = ["RELIANCE.NS", "HDFCBANK.NS", "INFY.NS", "TCS.NS", "ICICIBANK.NS", 
                   "HINDUNILVR.NS", "KOTAKBANK.NS", "LT.NS", "SBIN.NS", "AXISBANK.NS", 
                   "BAJFINANCE.NS", "BHARTIARTL.NS", "MARUTI.NS", "SUNPHARMA.NS", 
                   "TITAN.NS", "ULTRACEMCO.NS", "TATASTEEL.NS", "ASIANPAINT.NS", 
                   ]

# Download historical stock data for 3 years
df = yf.download(stocks, start="2021-01-01", end="2024-01-01")
df = df['Close']

# Step 2: Calculate Returns & Covariance Matrix
returns = df.pct_change().dropna()
expected_returns = returns.mean()
cov_matrix = returns.cov()

# Step 3: Construct QUBO Matrix
assets = len(expected_returns)
penalty = 1 / np.mean(abs(cov_matrix.values))  # Adjust for risk-return balance


ModuleNotFoundError: No module named 'qiskit_finance'

In [ ]:
tickers = ["RELIANCE.NS", "HDFCBANK.NS", "INFY.NS", "TCS.NS", "ICICIBANK.NS", 
                   "HINDUNILVR.NS", "KOTAKBANK.NS", "LT.NS", "SBIN.NS", "AXISBANK.NS", 
                   "BAJFINANCE.NS", "BHARTIARTL.NS", "MARUTI.NS", "SUNPHARMA.NS", 
                   "TITAN.NS", "ULTRACEMCO.NS", "TATASTEEL.NS", "ASIANPAINT.NS", 
                   ]
expected_returns = expected_returns[tickers].values
cov_matrix = cov_matrix.loc[tickers, tickers].values

print(expected_returns)
print(cov_matrix)

[5.78902168e-04 3.82466878e-04 4.80228247e-04 5.14112928e-04
 1.00856483e-03 2.89893917e-04 5.16867256e-05 1.50821582e-03
 1.33380358e-03 9.13062492e-04 6.36177939e-04 1.08379833e-03
 5.41478872e-04 1.14902014e-03 1.30495784e-03 1.06359204e-03
 2.88345206e-03 4.19013593e-04]
[[2.22453041e-04 7.81216911e-05 6.98640300e-05 5.75400172e-05
  8.36136645e-05 4.02240083e-05 7.73629130e-05 7.90079491e-05
  9.90500806e-05 8.52660018e-05 1.10792655e-04 6.38249395e-05
  5.97463222e-05 4.05535981e-05 7.64163498e-05 7.34133928e-05
  1.00126587e-04 4.99090041e-05]
 [7.81216911e-05 2.00683749e-04 6.88003730e-05 5.53348731e-05
  1.20675107e-04 3.98974740e-05 1.17869098e-04 8.72394035e-05
  1.12402414e-04 1.00214137e-04 1.19680381e-04 6.67055064e-05
  6.44247043e-05 4.27359540e-05 7.18448804e-05 8.01686029e-05
  5.57910645e-05 4.78850982e-05]
 [6.98640300e-05 6.88003730e-05 2.31265265e-04 1.38913402e-04
  6.66239292e-05 4.80695706e-05 4.75061147e-05 8.54027997e-05
  5.67928261e-05 5.45652755e-05 8.7251

In [ ]:
# q = 0.5  # set risk factor
budget = 16  # set budget

portfolio = PortfolioOptimization(
    expected_returns=expected_returns, covariances=cov_matrix, risk_factor=penalty, budget=budget
)
qp = portfolio.to_quadratic_program()

In [ ]:
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [ ]:
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), reps=10)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

print(result)
print("Optimal Portfolio Selection:", result.x)
print("Optimal Portfolio Value:", result.fval)

print("\n\n")
print_result(result)

fval=256.03610805651795, x_0=0.0, x_1=1.0, x_2=1.0, x_3=1.0, x_4=0.0, x_5=1.0, x_6=1.0, x_7=1.0, x_8=1.0, x_9=1.0, x_10=1.0, x_11=1.0, x_12=1.0, x_13=1.0, x_14=1.0, x_15=1.0, x_16=1.0, x_17=1.0, status=SUCCESS
Optimal Portfolio Selection: [0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Optimal Portfolio Value: 256.03610805651795



Optimal: selection [0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], value 256.0361

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 1 1 0 0 0 1 1 0 0 1 0 0 1 0 0 1 1]	85.3566		0.0020
[0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0]	87.4357		0.0020
[0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0]	68.4016		0.0020
[0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 1 1]	122.3830		0.0020
[1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0 0 0]	105.3669		0.0020
[1 1 0 0 1 1 1 0 0 0 0 0 1 0 1 1 0 1]	82.7069		0.0020
[1 0 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 1]	136.2060		0.0010
[0 1 0 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0]	102.9643		0.00